In [11]:
import numpy as np
import pandas as pd
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
   

In [12]:
def ChinaDaily_WebData(part,p_num = "Default", lan = "EN"):
 
    link_list = []
    name_list = []
    time_list = []
    my_url = "http://www.chinadaily.com.cn/"+ part
    uClient = uReq(my_url)
    page_html = uClient.read()
    uClient.close()

    # doing the html parsing
    page_soap = soup(page_html, "html.parser")
    
    # get the number of the pages
    try:
        if p_num == "Default":
            pages = page_soap.findAll("div",{"selectpage res-m"})
            page = pages[0].span.a.text
            page_num = page[21:]
        else:
            page_num = p_num 
        
    except:
        if p_num == "Default":
            pages = page_soap.findAll("div",{"selectpage"})
            page = pages[0].span.a.text
            page_num = page[21:]
        else:
            page_num = p_num
        
        
       

    #I have to count mannully for the number of the page
    for i in range(1,int(page_num)+1):
        my_url = "http://www.chinadaily.com.cn/"+ part + "page_" +str(i) + ".html"
        uClient = uReq(my_url)
        page_html = uClient.read()
        uClient.close()
        # doing the html parsing
        page_soap = soup(page_html, "html.parser")
        #grab each product
        containers = page_soap.findAll("div",{"mb10 tw3_01_2 "})

        for container in containers:
            link_to_article = container.h4.a["href"][2:]
            link_list.append(link_to_article)
            name_of_article = container.h4.a.text
            name_list.append(name_of_article)
            time_of_article = container.b.text
            time_list.append(time_of_article)

    author_list = [] 
    content_list = []
    for link in link_list:

        my_url = "http://" + link
        uClient = uReq(my_url)
        page_html = uClient.read()
        uClient.close()
        # doing the html parsing
        page_soap = soup(page_html, "html.parser")
        #get the authors name
        info = page_soap.find("div",{"info"})
        try:
            info_list = info.text.split("|")
            author = info_list[0]
            author_list.append(author)
        except:
            author_list.append(np.nan)


        try:
            #if there is a parsing for filping pages
            num_content = page_soap.find("span",{"pageno"})
            #get the number of pages
            num=num_content.a.text.split("\n")[1].split("/")[-1]


            #get the content of the webpage
            words = ""
            # going through each pages of the article
            for i in range(1,int(num)+1):

                my_url = "http://" + link[:-5] +"_"+str(i)+".html"


                uClient = uReq(my_url)
                page_html = uClient.read()
                uClient.close()
                # doing the html parsing
                page_soap = soup(page_html, "html.parser")

                content = page_soap.findAll("p",)
                content = content[:-1]
                for item in content:
                    words += item.text



        except:
            words = ""
            content = page_soap.findAll("p",)
            content = content[:-1]
            for item in content:
                words += item.text



        #pend the if the content has a next page
        #continue_run = page_soap.find("span",{"next"})


        content_list.append(words)

    df = pd.DataFrame({'title of the article': name_list,
                            'link of the article': link_list,
                            'author': author_list,
                            'content': content_list,                           
                            'time of the article': time_list})
    
    #clean the author column 
    df['author'] = df1['author'].map(lambda x: x.lstrip('\n\t By'))
    return df
    
   

## this function can scrap any part of the data from china daily
## just select the articles from which section.
## the example down below
## the result will return a pandas dataframe with the requested section of the article

In [13]:
df1 = ChinaDaily_WebData("/travel/guidesandtips/",1)

In [15]:
df1

,title of the article,link of the article,author,content,time of the article
0,Chinese homestay provider launches travel guide,www.chinadaily.com.cn/a/201906/03/WS5cf4bc2ca3...,\n\n By Yang Feiyue,Chinese homestay provider Sweetome has launche...,2019-06-03 14:20
1,The green experience,www.chinadaily.com.cn/a/201905/02/WS5cca4da5a3...,\n\n By Chen Meiling,A once barren town is now a haven for nature l...,2019-05-02 09:53
2,Discover intriguing animal species of China,www.chinadaily.com.cn/a/201904/16/WS5cb51dd1a3...,\n\n By Erik Nilsson,The giant panda is an international conservati...,2019-04-16 08:12
3,10 destinations you can't miss over Qingming F...,www.chinadaily.com.cn/a/201904/01/WS5ca184b0a3...,\n\n By Zhou Bing,"Qingming Festival, which falls on April 5 this...",2019-04-01 11:25
4,China's Great Wall turns green for St Patrick'...,www.chinadaily.com.cn/a/201903/18/WS5c8f3bd1a3...,\n\n By Zhang Xingjian,"China's winding Great Wall, for the sixth time...",2019-03-18 14:33
5,Young American helps create Chinese tourism vi...,www.chinadaily.com.cn/a/201901/02/WS5c2bfb63a3...,\n\n \n \t\t\t Xinhua,NEW YORK-Video producer and musician Justin Sc...,2019-01-02 07:44
6,Travel for Chinese New Year,www.chinadaily.com.cn/a/201812/28/WS5c25b6cba3...,\n\n By Li Wenrui,Less than two months away from the Spring Fest...,2018-12-28 13:38
7,What’s on in Hong Kong,www.chinadaily.com.cn/a/201812/12/WS5c10b2e9a3...,\n\n \n \t\t\t China Daily Asia,"Whether it's music, art, stage, screen, restau...",2018-12-12 15:04
8,The Pacific Islands: Meet your next favorite h...,www.chinadaily.com.cn/a/201904/03/WS5c00f3a8a3...,\n\n By Zhou Bing,It is expected that eight Pacific Island count...,2019-04-03 14:22
9,Expo plans tours for participants,www.chinadaily.com.cn/a/201811/10/WS5be65c62a3...,\n\n \n \t\t\t China Daily,The organizing committee of the China Internat...,2018-11-10 12:19


In [25]:
df1['author'] = df1['author'].map(lambda x: x.lstrip('\n\t By'))

In [26]:
df1

,title of the article,link of the article,author,content,time of the article,result
0,Chinese homestay provider launches travel guide,www.chinadaily.com.cn/a/201906/03/WS5cf4bc2ca3...,Yang Feiyue,Chinese homestay provider Sweetome has launche...,2019-06-03 14:20,Yang Feiyue
1,The green experience,www.chinadaily.com.cn/a/201905/02/WS5cca4da5a3...,Chen Meiling,A once barren town is now a haven for nature l...,2019-05-02 09:53,Chen Meiling
2,Discover intriguing animal species of China,www.chinadaily.com.cn/a/201904/16/WS5cb51dd1a3...,Erik Nilsson,The giant panda is an international conservati...,2019-04-16 08:12,Erik Nilsson
3,10 destinations you can't miss over Qingming F...,www.chinadaily.com.cn/a/201904/01/WS5ca184b0a3...,Zhou Bing,"Qingming Festival, which falls on April 5 this...",2019-04-01 11:25,Zhou Bing
4,China's Great Wall turns green for St Patrick'...,www.chinadaily.com.cn/a/201903/18/WS5c8f3bd1a3...,Zhang Xingjian,"China's winding Great Wall, for the sixth time...",2019-03-18 14:33,Zhang Xingjian
5,Young American helps create Chinese tourism vi...,www.chinadaily.com.cn/a/201901/02/WS5c2bfb63a3...,Xinhua,NEW YORK-Video producer and musician Justin Sc...,2019-01-02 07:44,Xinhua
6,Travel for Chinese New Year,www.chinadaily.com.cn/a/201812/28/WS5c25b6cba3...,Li Wenrui,Less than two months away from the Spring Fest...,2018-12-28 13:38,Li Wenrui
7,What’s on in Hong Kong,www.chinadaily.com.cn/a/201812/12/WS5c10b2e9a3...,China Daily Asia,"Whether it's music, art, stage, screen, restau...",2018-12-12 15:04,China Daily Asia
8,The Pacific Islands: Meet your next favorite h...,www.chinadaily.com.cn/a/201904/03/WS5c00f3a8a3...,Zhou Bing,It is expected that eight Pacific Island count...,2019-04-03 14:22,Zhou Bing
9,Expo plans tours for participants,www.chinadaily.com.cn/a/201811/10/WS5be65c62a3...,China Daily,The organizing committee of the China Internat...,2018-11-10 12:19,China Daily
